In [2]:
from pymongo import MongoClient
import re
import pandas as pd
import numpy as np

In [145]:
client = MongoClient('mongodb://localhost:27017')
database = client['Thesis']
collectionDB = database.get_collection("citations")
collectionDB2 = database.get_collection("citations_fixed")

In [21]:
dataCitations = pd.DataFrame()

In [22]:
print("Start  -------------------------------")

for document in collectionDB.find():
    number = document["file"]
    citations = document["classification"]
    data = []
    for citation in citations:
        data.append({'id_doc': number, 'item': citation})
    df = pd.DataFrame(data)
    dataCitations = pd.concat([df, dataCitations])

Start  -------------------------------


In [23]:
dataCitations

,id_doc,item
0,A123-22,AUTO123/2022
1,A123-22,AUTO288/2010
2,A123-22,AUTO368/2015
3,A123-22,C-174/2017
4,A123-22,C-049/2012
...,...,...
1,A019-92,C-416/1992
0,A018-92,
1,A018-92,C-416/1992
0,A017-92,AUTO017/1992


In [67]:
dataCitations2 = dataCitations

# Getting the info to standardize it

In [92]:
def trans_year(string:str):
    string = str(string)
    first_ch = string[0]
    if  first_ch in ["0", "1", "2"]:
        year_fixed = "20" + string
    else:
        year_fixed = "19" + string
    return year_fixed

In [103]:
def trans_type(string:str):
    string = str(string)
    if string[0] == 'S':
        return 'SU'
    else:
        return string[0]

In [95]:
typeSentence_re = r'^([a-zA-Z]+)'
numberSentece_re = r'(\d+)'
yearSentence_re = r'(\d+)$'

dataCitations2['id_doc_type'] = dataCitations2['id_doc'].str.extract(typeSentece_re)
dataCitations2['id_doc_type'] = dataCitations2['id_doc_type'].str.upper()
dataCitations2['id_doc_number'] = dataCitations2['id_doc'].str.extract(numberSentece_re)
dataCitations2['id_doc_number'] = dataCitations2['id_doc_number'].astype(str).str.zfill(4)
dataCitations2['id_doc_year'] = dataCitations2['id_doc'].str.extract(yearSentece_re)
dataCitations2['id_doc_year_fix'] = dataCitations2['id_doc_year'].apply(trans_year)

In [49]:
dataCitations2['cite_type'] = dataCitations2['item'].str.extract(typeSentece_re)
dataCitations2['cite_type'] = dataCitations2['cite_type'].str.upper()
dataCitations2['cite_number'] = dataCitations2['item'].str.extract(numberSentece_re)
dataCitations2['cite_number'] = dataCitations2['cite_number'].astype(str).str.zfill(4)
dataCitations2['cite_year'] = dataCitations2['item'].str.extract(yearSentece_re)

In [104]:
dataCitations2['cite_type_fixed'] = dataCitations2['cite_type'].apply(trans_type)

In [99]:
dataCitations2['type_id_doc'].value_counts()

T      194470
C       76383
A       49833
SU      11117
E           5
ACU         4
Name: type_id_doc, dtype: int64

In [106]:
dataCitations2['cite_type'].value_counts(dropna=False)

T        160939
C         87616
NaN       33535
AUTO      24725
SU        14506
A         10079
AUTOA       297
AT           53
AC           17
TO           16
CO           15
CN            6
AA            4
AN            1
CT            1
TC            1
TT            1
Name: cite_type, dtype: int64

In [105]:
dataCitations2['cite_type_fixed'].value_counts()

T     160957
C      87638
A      35176
n      33535
SU     14506
Name: cite_type_fixed, dtype: int64



## Fixing the id_doc


In [50]:
dataCitations2['id_doc_fixed'] = dataCitations2['type_id_doc'] + dataCitations2['id_doc_number']

,id_doc,item,type_id_doc,type_id_number,type_id_year,id_doc_type,id_doc_number,id_doc_year,cite_type,cite_number,cite_year
0,A123-22,AUTO123/2022,A,0123,22,A,0123,22,AUTO,0123,2022
1,A123-22,AUTO288/2010,A,0123,22,A,0123,22,AUTO,0288,2010
2,A123-22,AUTO368/2015,A,0123,22,A,0123,22,AUTO,0368,2015
3,A123-22,C-174/2017,A,0123,22,A,0123,22,C,0174,2017
4,A123-22,C-049/2012,A,0123,22,A,0123,22,C,0049,2012
...,...,...,...,...,...,...,...,...,...,...,...
1,A019-92,C-416/1992,A,0019,92,A,0019,92,C,0416,1992
0,A018-92,,A,0018,92,A,0018,92,NaN,0nan,NaN
1,A018-92,C-416/1992,A,0018,92,A,0018,92,C,0416,1992
0,A017-92,AUTO017/1992,A,0017,92,A,0017,92,AUTO,0017,1992


In [107]:
dataCitations2['id_doc_fixed'] = dataCitations2['type_id_doc']+ "-" + dataCitations2['id_doc_number'] + "/" + dataCitations2['id_doc_year_fix']
dataCitations2['id_cite_fixed'] = dataCitations2['cite_type_fixed']+ "-" + dataCitations2['cite_number'] + "/" + dataCitations2['cite_year']

In [108]:
dataCitations2

,id_doc,item,type_id_doc,type_id_number,type_id_year,id_doc_type,id_doc_number,id_doc_year,cite_type,cite_number,cite_year,id_doc_fixed,id_doc_year_fix,cite_type_fixed,id_cite_fixed
0,A123-22,AUTO123/2022,A,0123,22,A,0123,22,AUTO,0123,2022,A-0123/2022,2022,A,A-0123/2022
1,A123-22,AUTO288/2010,A,0123,22,A,0123,22,AUTO,0288,2010,A-0123/2022,2022,A,A-0288/2010
2,A123-22,AUTO368/2015,A,0123,22,A,0123,22,AUTO,0368,2015,A-0123/2022,2022,A,A-0368/2015
3,A123-22,C-174/2017,A,0123,22,A,0123,22,C,0174,2017,A-0123/2022,2022,C,C-0174/2017
4,A123-22,C-049/2012,A,0123,22,A,0123,22,C,0049,2012,A-0123/2022,2022,C,C-0049/2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,A019-92,C-416/1992,A,0019,92,A,0019,92,C,0416,1992,A-0019/1992,1992,C,C-0416/1992
0,A018-92,,A,0018,92,A,0018,92,NaN,0nan,NaN,A-0018/1992,1992,n,NaN
1,A018-92,C-416/1992,A,0018,92,A,0018,92,C,0416,1992,A-0018/1992,1992,C,C-0416/1992
0,A017-92,AUTO017/1992,A,0017,92,A,0017,92,AUTO,0017,1992,A-0017/1992,1992,A,A-0017/1992


In [109]:
dataCitationsFixed = dataCitations2[["id_doc_fixed", "id_cite_fixed"]]

In [110]:
dataCitationsFixed

,id_doc_fixed,id_cite_fixed
0,A-0123/2022,A-0123/2022
1,A-0123/2022,A-0288/2010
2,A-0123/2022,A-0368/2015
3,A-0123/2022,C-0174/2017
4,A-0123/2022,C-0049/2012
...,...,...
1,A-0019/1992,C-0416/1992
0,A-0018/1992,NaN
1,A-0018/1992,C-0416/1992
0,A-0017/1992,A-0017/1992


In [112]:
list_jur =  dataCitationsFixed['id_doc_fixed'].unique().tolist()


In [123]:
dataCitationsFixed_filter = dataCitationsFixed[dataCitationsFixed['id_cite_fixed'].isin(list_jur)]

In [126]:
dataCitationsFixed_filter['compare'] = (dataCitationsFixed_filter['id_doc_fixed'] == dataCitationsFixed_filter['id_cite_fixed']).astype(int)


C:\Users\anama\AppData\Local\Temp\ipykernel_55648\2100049790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataCitationsFixed_filter['compare'] = (dataCitationsFixed_filter['id_doc_fixed'] == dataCitationsFixed_filter['id_cite_fixed']).astype(int)


In [128]:
dataCitationsFixed_filter = dataCitationsFixed_filter.loc[dataCitationsFixed_filter['compare'] == 0]

In [129]:
dataCitationsFixed_filter

,id_doc_fixed,id_cite_fixed,compare
1,A-0123/2022,A-0288/2010,0
2,A-0123/2022,A-0368/2015,0
3,A-0123/2022,C-0174/2017,0
4,A-0123/2022,C-0049/2012,0
5,A-0123/2022,C-0972/2004,0
...,...,...,...
1,A-0022/1992,C-0416/1992,0
1,A-0021/1992,C-0416/1992,0
1,A-0020/1992,C-0416/1992,0
1,A-0019/1992,C-0416/1992,0


In [116]:
dataCitationsFixed_disconn = dataCitationsFixed[dataCitationsFixed['id_cite_fixed'].isna()]

In [117]:
dataCitationsFixed_disconn

,id_doc_fixed,id_cite_fixed
4,A-1211/2022,NaN
2,A-1165/2022,NaN
1,A-0115/2022,NaN
11,A-1143/2022,NaN
1,A-1135/2022,NaN
...,...,...
0,A-0021/1992,NaN
0,A-0020/1992,NaN
0,A-0019/1992,NaN
0,A-0018/1992,NaN


In [130]:
consolCitation = pd.concat([dataCitationsFixed_filter, dataCitationsFixed_disconn])

In [131]:
consolCitation

,id_doc_fixed,id_cite_fixed,compare
1,A-0123/2022,A-0288/2010,0.0
2,A-0123/2022,A-0368/2015,0.0
3,A-0123/2022,C-0174/2017,0.0
4,A-0123/2022,C-0049/2012,0.0
5,A-0123/2022,C-0972/2004,0.0
...,...,...,...
0,A-0021/1992,NaN,NaN
0,A-0020/1992,NaN,NaN
0,A-0019/1992,NaN,NaN
0,A-0018/1992,NaN,NaN


In [134]:
def countkey(df, column):
    counting = df.groupby(column).size().reset_index(name='count')
    df_count = pd.merge(df, counting, on=column)
    return df_count

In [135]:
consolCitation2 = countkey(consolCitation, 'id_doc_fixed')

In [136]:
consolCitation2

,id_doc_fixed,id_cite_fixed,compare,count
0,A-0123/2022,A-0288/2010,0.0,8
1,A-0123/2022,A-0368/2015,0.0,8
2,A-0123/2022,C-0174/2017,0.0,8
3,A-0123/2022,C-0049/2012,0.0,8
4,A-0123/2022,C-0972/2004,0.0,8
...,...,...,...,...
272891,C-0477/1992,NaN,NaN,1
272892,C-0447/1992,NaN,NaN,1
272893,C-0221/1992,NaN,NaN,1
272894,C-0004/1992,NaN,NaN,1


In [137]:
consolCitation3 = consolCitation2.loc[(consolCitation2['id_cite_fixed'].notna()) | ((consolCitation2['id_cite_fixed'].isna()) & (consolCitation2['count'] == 1))]

In [139]:
consolCitation3 = consolCitation3[["id_doc_fixed", "id_cite_fixed"]]

In [140]:
list_jur_fin =  consolCitation3['id_doc_fixed'].unique().tolist()

In [ ]:
for iSentence in list_jur_fin:
    data = consolCitation3.loc[consolCitation3['id_doc_fixed'] == iSentence]
    res = dict()
    res['version'] = "0.0.1"
    res['id_doc'] = iSentence
    res['citations'] = data['id_cite_fixed'].unique().tolist()
    result_object = collectionDB2.insert_one(res)